# Postgres + Pandas

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import os

### Store CSV into DataFrame

In [2]:
csv_file = os.path.join("Resources", "revenue.csv")
movies_df = pd.read_csv(csv_file)
movies_df.shape

(6, 3)

In [3]:
#Creating a copy of the df to choose only required columns

df=revenue_df[['Category','Revenue (in millions) $
','gamers']].copy()

In [4]:
df.head()

,Title,Netflix,Hulu,Prime Video
0,Inception,1,0,0
1,The Matrix,1,0,0
2,Avengers: Infinity War,1,0,0
3,Back to the Future,1,0,0
4,"The Good, the Bad and the Ugly",1,0,1


In [5]:
df.dtypes

Title          object
Netflix         int64
Hulu            int64
Prime Video     int64
dtype: object

### Summarizing streaming channels into a single column

In [7]:
lisst=[]
for index,row in df.iterrows():
    if (row[1]==1) and (row[2]==0) and (row[3]==0):
        lisst.append('Netflix')
    elif (row[1]==0) and (row[2]==1) and (row[3]==0):
        lisst.append('Hulu')
    elif row[1]==0 and row[2]==0 and row[3]==1:
        lisst.append('Prime Video')   
    elif row[1]==1 and row[2]==1 and row[3]==0:
        lisst.append(('Netflix','Hulu'))
    elif row[1]==1 and row[2]==0 and row[3]==1:
        lisst.append(('Netflix','Prime Video'))
    elif row[1]==0 and row[2]==1 and row[3]==1:
        lisst.append(('Hulu','Prime Video'))
    elif row[1]==1 and row[2]==1 and row[3]==1:
        lisst.append(('Netflix','Hulu','Prime Video'))
    else:
        lisst.append('Disney')
    
    
            

In [9]:
#Adding the newly created list to the df
df['Streaming_Channel'] = lisst

In [10]:
df.head()

,Title,Netflix,Hulu,Prime Video,Streaming_Channel
0,Inception,1,0,0,Netflix
1,The Matrix,1,0,0,Netflix
2,Avengers: Infinity War,1,0,0,Netflix
3,Back to the Future,1,0,0,Netflix
4,"The Good, the Bad and the Ugly",1,0,1,"(Netflix, Prime Video)"


In [11]:
#Droping all non required columns
df_new=df[['Title','Streaming_Channel']].copy()

In [12]:
df_new.columns=('title','streaming_channel')

### Store JSON data into a DataFrame

In [ ]:
json_file = os.path.join("..", "Resources", "customer_location.json")
customer_location_df = pd.read_json(json_file)
customer_location_df.head()

### Clean DataFrame

In [ ]:
#Creating a copy only for the required column

new_customer_location_df = customer_location_df[["id", "address", "us_state"]].copy()
new_customer_location_df.head()

### Connect to local database

In [14]:
pg_user = 'postgres'
pg_password = 'postgres'
db_name = 'ETL Project'

connection_string = f"{pg_user}:Jennifer11@localhost:5432/{db_name}"
engine = create_engine(f'postgresql://{connection_string}')

### Check for tables

In [15]:
engine.table_names()

['movies', 'wiki']

### Use pandas to load csv converted DataFrame into database

In [16]:
wiki_file=os.path.join('Resources','wiki.csv')
wiki_df=pd.read_csv(wiki_file)

In [17]:
wiki_df.head()

,film,director
0,Citizen Kane,Orson Welles
1,Casablanca,Michael Curtiz
2,The Godfather,Francis Ford Coppola
3,Gone with the Wind,Victor Fleming
4,Lawrence of Arabia,David Lean


In [18]:
df_new.to_sql(name='movies', con=engine, if_exists='append', index=False)

### Use pandas to load json converted DataFrame into database

In [19]:
wiki_df.to_sql(name='wiki', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the customer_name table
* NOTE: can also check using pgAdmin

In [21]:
pd.read_sql_query('select * from movies', con=engine).head()

,title,streaming_channel
0,Inception,Netflix
1,The Matrix,Netflix
2,Avengers: Infinity War,Netflix
3,Back to the Future,Netflix
4,"The Good, the Bad and the Ugly","(Netflix,""Prime Video"")"


In [ ]:
pd.read_sql_query('select * from wiki', con=engine).head()